<a href="https://colab.research.google.com/github/albertsalgueda/tensorflow/blob/main/Deep_Reinforced_Learning_for_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%208%20-%20Reinforcement%20Learning%20para%20problemas%20de%20Stock%20Market%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

     |████████████████████████████████| 109 kB 12.1 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


## Paso 2: Importar las dependencias del proyecto

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [ ]:
tf.__version__

'2.8.0'

## Paso 3: Construir la red neuronal de la AI del Trader 

In [ ]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Manten, Compra, Vende
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [ ]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [ ]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [ ]:
df = data_reader.DataReader("AAPL",data_source ="yahoo")

In [ ]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-02-22,34.279999,34.027500,34.107498,34.277500,83347600.0,32.363441
2017-02-23,34.369999,34.075001,34.345001,34.132500,83152800.0,32.226536
2017-02-24,34.165001,33.820000,33.977501,34.165001,87106400.0,32.257229
2017-02-27,34.360001,34.070000,34.285000,34.232498,81029600.0,32.320957
2017-02-28,34.360001,34.174999,34.270000,34.247501,93931600.0,32.335114
...,...,...,...,...,...,...
2022-02-14,169.580002,166.559998,167.369995,168.880005,86185500.0,168.880005
2022-02-15,172.949997,170.250000,170.970001,172.789993,64286300.0,172.789993
2022-02-16,173.339996,170.050003,171.850006,172.550003,61177400.0,172.550003


In [ ]:
df.index

DatetimeIndex(['2017-02-22', '2017-02-23', '2017-02-24', '2017-02-27',
               '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03',
               '2017-03-06', '2017-03-07',
               ...
               '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-11', '2022-02-14', '2022-02-15', '2022-02-16',
               '2022-02-17', '2022-02-18'],
              dtype='datetime64[ns]', name='Date', length=1259, freq=None)

### State creator

In [ ]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Cargar una divisa de mercado

In [ ]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [ ]:
data.tail()

Date
2022-02-14    168.880005
2022-02-15    172.789993
2022-02-16    172.550003
2022-02-17    168.880005
2022-02-18    167.300003
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [ ]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Definir el modelo del AI Trader

In [ ]:
trader = AI_Trader(window_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episodio: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    print(t)
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Compra
      trader.inventory.append(data[t])
      print("AI Trader compró: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Vende
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

Episodio: 1/1000


  0%|          | 0/1258 [00:00<?, ?it/s]

0
AI Trader compró:  $ 34.277500
1
AI Trader compró:  $ 34.132500
2
AI Trader compró:  $ 34.165001
3
AI Trader vendió:  $ 34.232498  Beneficio: - $ 0.045002
4
5
AI Trader vendió:  $ 34.947498  Beneficio: $ 0.814999
6
AI Trader compró:  $ 34.740002
7
AI Trader vendió:  $ 34.945000  Beneficio: $ 0.779999
8
AI Trader compró:  $ 34.834999
9
AI Trader compró:  $ 34.880001
10
AI Trader compró:  $ 34.750000
11
AI Trader vendió:  $ 34.669998  Beneficio: - $ 0.070004
12
AI Trader vendió:  $ 34.785000  Beneficio: - $ 0.049999
13
14
15
16
17
AI Trader compró:  $ 34.997501
18
AI Trader compró:  $ 35.365002
19
20
AI Trader vendió:  $ 35.355000  Beneficio: $ 0.474998
21
AI Trader vendió:  $ 35.230000  Beneficio: $ 0.480000
22
AI Trader vendió:  $ 35.160000  Beneficio: $ 0.162498
23
24
25
26
AI Trader vendió:  $ 35.982498  Beneficio: $ 0.617496
27
28
AI Trader compró:  $ 35.924999
29
AI Trader compró:  $ 36.192501
30
AI Trader vendió:  $ 36.005001  Beneficio: $ 0.080002
31
AI Trader compró:  $ 35.915

  3%|▎         | 33/1258 [00:06<03:55,  5.19it/s]

33
AI Trader vendió:  $ 35.792500  Beneficio: - $ 0.400002


...

921
AI Trader vendió:  $ 115.980003  Beneficio: $ 26.262505


 73%|███████▎  | 922/1258 [59:52<22:38,  4.04s/it]

922
AI Trader compró:  $ 117.510002


 73%|███████▎  | 923/1258 [59:56<22:26,  4.02s/it]

923
AI Trader vendió:  $ 116.870003  Beneficio: $ 25.237503
